In [2]:
import pandas as pd
import numpy as np
import os
import glob as gb
import sys
sys.path.insert(0, "../../")
from clases.bd.conexion2 import MyDatabase2
conn = MyDatabase2() 

Conexion exitosa con la Base de datos:irvin_hisminsa


In [3]:
def consulta_rvalidacion(conn,grupo):
    paquete=conn.df("""SELECT * FROM e_vida.reglas_validacion rv WHERE st=1 AND id=%s """ % grupo)   
    return paquete

def consulta_esquema(conn,id):
    paquete=conn.df("""SELECT * FROM e_vida.esquema e WHERE  st=1 AND id_reglas_validacion=%s ORDER BY e.id"""%id)   
    return paquete 

def consulta_codigo(conn,id):
    paquete=conn.df("""SELECT * FROM e_vida.codigos e WHERE  st=1 AND id_esquema=%s"""%id)   
    return paquete 

def consulta_ultima_atencion(conn):
    paquete=conn.df("""WITH atenciones_ordenadas AS (  SELECT mhtd.abrev_tipo_doc, mp.numero_documento, nt.fecha_atencion, nt.id_establecimiento, ROW_NUMBER() OVER (PARTITION BY mp.id_tipo_documento, mp.numero_documento
ORDER BY nt.fecha_atencion DESC) AS rn
FROM maestros.nominal_trama2 nt
INNER JOIN maestros.maestro_paciente mp ON mp.id_paciente = nt.id_paciente
INNER JOIN maestros.maestro_his_tipo_doc mhtd ON mhtd.id_tipo_documento =mp.id_tipo_documento 
)
SELECT abrev_tipo_doc, numero_documento, id_establecimiento
FROM atenciones_ordenadas
WHERE rn = 1;""")   
    return paquete 
u_atencion=consulta_ultima_atencion(conn)

d:\Irvin\Irvin\Python\Python-GERESA\modulos\gestante\../..\clases\bd\conexion2.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)


In [4]:
data = {'abrev_tipo_doc': [],
        'numero_documento': [],
        'genero': [],
        'fecha_nacimiento': [],
        'edad': [] }
df = pd.DataFrame(data)


df_rvalidacion = consulta_rvalidacion(conn,2)

for index, row in df_rvalidacion.iterrows():
    id_p = row['id']
    inicio = row['inicio']
    fin = row['fin']

     #creamos una lista vacia para almacenas los id
    id_col= []
    df_esquema = pd.DataFrame({})
    #print(id_p)
    df_esquema=consulta_esquema(conn,id_p)

    for h, row1 in df_esquema.iterrows():
        id_es = row1['id']
        cantidad_es = row1['cantidad']

        #crea nuevo data frama pequeño 
        df_min = pd.DataFrame(data)
        #consulta SQL para jalar los codigos 
        df_codigo= consulta_codigo(conn,id_es)

        for i, row2 in df_codigo.iterrows():
            dx = row2['codigo_item']
            tdx = row2['tipo_diagnostico']
            lab1="AND nt.lab1='"+row2['lab_1']+"'" if row2['lab_1'] is not None  else ''
            lab2="AND nt.lab2='"+row2['lab_2']+"'" if row2['lab_2'] is not None  else ''
            lab3="AND nt.lab3='"+row2['lab_3']+"'" if row2['lab_3'] is not None  else '' 
            #print(1)

            sql = """ SELECT DISTINCT  mhtd.abrev_tipo_doc,mp.numero_documento, mp.genero ,mp.fecha_nacimiento  ,TO_CHAR(AGE(NOW(), fecha_nacimiento), 'YY "años" MM "meses" DD "días"') AS edad  ,  nt.fecha_atencion ,age(nt.fecha_atencion,mp.fecha_nacimiento) as tiempo_%s  FROM maestros.nominal_trama2 nt
INNER JOIN maestros.maestro_paciente mp ON mp.id_paciente =nt.id_paciente
INNER JOIN maestros.maestro_his_tipo_doc mhtd ON mhtd.id_tipo_documento =mp.id_tipo_documento  
 WHERE nt.codigo_item ='%s' AND nt.tipo_diagnostico='%s' %s  """ % (i+1,dx,tdx, lab1)
            #print(sql)
            sql = conn.df(sql)
            ####============================ configuracion de Nombre ========================================
            lab1= "|"+row2['lab_1'] if row2['lab_1'] is not None  else ''
            lab2= "|"+row2['lab_2'] if row2['lab_2'] is not None  else ''
            lab3= "|"+row2['lab_3'] if row2['lab_3'] is not None  else ''
            head = dx.replace('.', '_')
            head = """%s|%s%s%s%s"""%(head,tdx,lab1,lab2,lab3)
            sql.rename(columns={'fecha_atencion': head}, inplace=True)
            ####================================================================================================
            #  head_esquema.append(head)
            ###========================================| unir con nuevo data frame |===========================
            sql = sql.set_index(['abrev_tipo_doc','numero_documento','genero','fecha_nacimiento','edad'])
            sql = sql.groupby(sql.index).tail(1)
            
            df_min = pd.merge(df_min, sql, on=['abrev_tipo_doc','numero_documento','genero','fecha_nacimiento','edad'], how='outer')
            df_min.reset_index() 
            #df = pd.concat([df, sql], axis=0, ignore_index=True)       
       
        # Columnas a contar
        #hh="id_"+str(id_es)  
        #id_col.append(hh)     
        #df_min[hh] = df_min[head_esquema].count(axis=1)
        #df_min[hh] = df_min.count(axis=1) -1

        # Convertir la columna a tipo string
        #df_min.loc[df_min[hh] >= cantidad_es, hh] = 'Si'
        
        df = pd.merge(df, df_min, on=['abrev_tipo_doc','numero_documento','genero','fecha_nacimiento','edad'], how='outer')


        #print(df)
        #df.rename(columns={'col': }, inplace=True)
    #df[id_col] = df[id_col].fillna(0)
    # verificar la cantidad de 'Si'
    #df['total'] = df.apply(lambda row: sum(row[col] == 'Si' for col in id_col), axis=1)
    #df[id_col] = df[id_col].astype(str)
        
        

df.head(10)

d:\Irvin\Irvin\Python\Python-GERESA\modulos\gestante\../..\clases\bd\conexion2.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)
d:\Irvin\Irvin\Python\Python-GERESA\modulos\gestante\../..\clases\bd\conexion2.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)
d:\Irvin\Irvin\Python\Python-GERESA\modulos\gestante\../..\clases\bd\conexion2.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)


,abrev_tipo_doc,numero_documento,genero,fecha_nacimiento,edad,99381_01|D|1,tiempo_1,99381_01|D|2,tiempo_2,99381_01|D|3,tiempo_3,99381_01|D|4,tiempo_4
0,CNV,10178977,M,2022-12-24,01 años 00 meses 03 días,2023-01-04,11 days,NaN,NaT,NaN,NaT,NaN,NaT
1,CNV,10276542,M,2023-03-23,00 años 09 meses 04 días,2023-03-31,8 days,2023-04-03,11 days,2023-04-10,18 days,NaN,NaT
2,CNV,10451860,M,2023-10-04,00 años 02 meses 23 días,2023-10-09,5 days,NaN,NaT,NaN,NaT,NaN,NaT
3,CNV,25210050,F,2023-07-13,00 años 05 meses 14 días,2023-07-17,4 days,NaN,NaT,NaN,NaT,NaN,NaT
4,CNV,2528041,M,2023-01-27,00 años 11 meses 00 días,2023-01-30,3 days,2023-02-03,7 days,2023-02-10,14 days,NaN,NaT
5,CNV,2528042,F,2023-01-31,00 años 10 meses 27 días,2023-02-02,2 days,2023-02-12,12 days,2023-02-18,18 days,NaN,NaT
6,CNV,2528043,M,2023-02-03,00 años 10 meses 24 días,2023-02-06,3 days,2023-02-10,7 days,NaN,NaT,NaN,NaT
7,CNV,2528045,F,2023-02-05,00 años 10 meses 22 días,2023-02-08,3 days,NaN,NaT,NaN,NaT,NaN,NaT
8,CNV,81744177,M,2023-03-02,00 años 09 meses 25 días,2023-03-07,5 days,2023-03-10,8 days,2023-03-16,14 days,2023-03-23,21 days
9,CNV,82300001,M,2023-10-27,00 años 02 meses 00 días,2023-11-02,6 days,2023-11-07,11 days,2023-11-14,18 days,2023-11-21,25 days


In [ ]:
df.to_excel('Plan_Materno.xlsx', index=False)

In [4]:
his=conn.df("""SELECT DISTINCT mp.numero_documento  FROM maestros.nominal_trama nt
LEFT JOIN maestros.maestro_paciente mp ON mp.id_paciente =nt.id_paciente 
WHERE (nt.codigo_item  LIKE ANY (ARRAY['Z349%', 'Z359%']) OR  nt.id_otra_condicion =1)""") 

d:\Irvin\Irvin\Python\pandas\modulos\gestante\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)


In [4]:
cnv=conn.df("""SELECT c.nu_doc_madre FROM maestros.cnv c WHERE  EXTRACT(YEAR FROM c.fe_nacido) = 2023""") 

d:\Irvin\Irvin\Python\pandas\modulos\gestante\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)


In [9]:
df1_sin_df2 = his[~his.isin(cnv.to_dict(orient='list')).all(axis=1)]
print(df1_sin_df2)

      numero_documento
0           0005622630
1            000892044
2           0010412094
3            001120005
4            001155036
...                ...
22202       SD-0004149
22203       SD-0004698
22204       SD-5371784
22205       SD-7184454
22206             None

[22207 rows x 1 columns]


In [6]:
nombre_funcion = "nombre_de_tu_funcion"  # Reemplaza con el nombre de tu función
parametros = [1, 2, 3]  # Reemplaza con tus parámetros

# Obtén la función utilizando el nombre almacenado en la cadena
funcion = globals().get(nombre_funcion)

# Verifica si la función existe
if funcion is not None and callable(funcion):
    # Invoca la función con los parámetros
    resultado = funcion(*parametros)
    print("Resultado de la función:", resultado)
else:
    print("La función '{}' no existe o no es llamable.".format(nombre_funcion))


La función 'nombre_de_tu_funcion' no existe o no es llamable.


: 